
# TARGET_AVG_COST_SHARE Robustness
Vary `TARGET_AVG_COST_SHARE` while holding $	au_d=0.2$ and $	au_f=0.5$ fixed. Outputs are written to `results/robustness`.


In [ ]:

import random
from pathlib import Path
from decimal import Decimal

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from tqdm.auto import tqdm

from optimal_ipr.distributions import value_distribution, build_theta_distribution
from optimal_ipr.probability import build_subjective_probability
from optimal_ipr.cost import build_cost_function
from optimal_ipr.fee import build_fee_schedule
from optimal_ipr.lookup import build_lookup_tables
from optimal_ipr.outcomes import welfare_outcomes

np.random.seed(0)
random.seed(0)

RESULTS_DIR = Path("results/robustness")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)


def ensure_exists(path: Path) -> Path:
    """Ensure parent directory exists for the given path and return the path."""
    path.parent.mkdir(parents=True, exist_ok=True)
    return path


In [ ]:

from dataclasses import dataclass
from typing import Any, Dict


def utilitarian(th):
    return 1.0

GOVERNMENT_PREFERENCES = {"utilitarian": utilitarian}
REGULATOR_PREFERENCES = {"welfarist_balanced": {"phi": 0.5, "psi": 0.5}}


@dataclass(frozen=True)
class Context:
    v_grid: np.ndarray
    v_weights: np.ndarray
    f: Any
    F: Any
    F_inv: Any
    p: Any
    Z: Any
    tau_d: float
    tau_f: float
    bar_beta_grid: np.ndarray
    gov_prefs: Dict[str, Any]
    reg_prefs: Dict[str, Any]


def build_context() -> Context:
    """Build invariant components of the model."""
    v_grid, v_weights = value_distribution(n_v=51, sigma=3.0)
    f, F, F_inv = build_theta_distribution(noise_level=0.20)
    p = build_subjective_probability(base_k=1.25, m_comp=5, F=F, F_inv=F_inv)
    Z = build_fee_schedule(zeta=0.04, fee_M=1.5)
    tau_d = 0.2
    tau_f = 0.5
    bar_beta_grid = np.linspace(0.0, 1.0, 101)
    return Context(v_grid, v_weights, f, F, F_inv, p, Z, tau_d, tau_f, bar_beta_grid,
                   GOVERNMENT_PREFERENCES, REGULATOR_PREFERENCES)

ctx = build_context()


In [ ]:


def run_for_share(share: float, ctx: Context) -> dict:
    """Run government optimization for a given TARGET_AVG_COST_SHARE."""
    c = build_cost_function(ctx.f, TARGET_AVG_COST_SHARE=share, C_MIN_COST=0.05, GAMMA_C_COST=3.0)
    F_scalar = lambda t: float(ctx.F(np.array([t])))
    theta_tilde_table, theta_winner_table, *_ = build_lookup_tables(
        ctx.p, c, ctx.Z, ctx.f, F_scalar, ctx.tau_d, ctx.tau_f, ctx.bar_beta_grid, ctx.v_grid, seed=42
    )
    results_table = welfare_outcomes(
        tau_d=ctx.tau_d,
        tau_f=ctx.tau_f,
        gov_prefs=ctx.gov_prefs,
        reg_prefs=ctx.reg_prefs,
        v_grid=ctx.v_grid,
        v_weights=ctx.v_weights,
        theta_tilde_table=theta_tilde_table,
        theta_winner_table=theta_winner_table,
        f=ctx.f,
        F=ctx.F,
        F_inv=ctx.F_inv,
        p=ctx.p,
        c=c,
        Z=ctx.Z,
        feas=False,
    )
    if results_table.empty:
        raise ValueError("welfare_outcomes returned an empty table")
    bar_beta_opt = float(results_table.loc[0, 'Optimal Policy'])
    welfare_pct_change = float(results_table.loc[0, 'Welfare % Change'])
    return {
        'target_avg_cost_share': share,
        'bar_beta_opt': bar_beta_opt,
        'welfare_pct_change': welfare_pct_change,
    }


In [ ]:

from time import time

shares = [round(0.10 + 0.05 * i, 2) for i in range(14)]
start_time = time()
results = Parallel(n_jobs=-1, backend="loky")(
    delayed(run_for_share)(s, ctx) for s in tqdm(shares)
)
total_time = time() - start_time
results_df = pd.DataFrame(results).sort_values("target_avg_cost_share").reset_index(drop=True)


In [ ]:

csv_path = ensure_exists(RESULTS_DIR / "target_avg_cost_share_sweep.csv")
results_df.to_csv(csv_path, index=False)

plt.rcParams.update({"font.size": 14})
fig, ax1 = plt.subplots(figsize=(16, 9))
color1, color2 = "tab:blue", "tab:orange"
ax1.set_xlabel("Target Average Cost Share")
ax1.set_ylabel(r"Optimal $\overline{\beta}$", color=color1)
ax1.plot(results_df["target_avg_cost_share"], results_df["bar_beta_opt"], marker="o", color=color1, label=r"Optimal $\overline{\beta}$")
ax2 = ax1.twinx()
ax2.set_ylabel("Welfare % Change", color=color2)
ax2.plot(results_df["target_avg_cost_share"], results_df["welfare_pct_change"], marker="s", color=color2, label="Welfare % Change")
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
caption = "Robustness sweep with tau_d=0.2 and tau_f=0.5"
fig.suptitle("TARGET_AVG_COST_SHARE Robustness")
fig.text(0.5, -0.05, caption, ha="center")
png_path = ensure_exists(RESULTS_DIR / "target_avg_cost_share_sweep.png")
fig.savefig(png_path, dpi=100, bbox_inches="tight")
plt.show()


In [ ]:

baseline_share = 0.50
baseline_calc = run_for_share(baseline_share, ctx)
baseline_row = results_df.loc[results_df['target_avg_cost_share'] == baseline_share].iloc[0]

tol_beta = 1e-6
tol_welfare = 1e-6
beta_diff = abs(baseline_calc['bar_beta_opt'] - baseline_row['bar_beta_opt'])
welfare_diff = abs(baseline_calc['welfare_pct_change'] - baseline_row['welfare_pct_change'])

if beta_diff <= tol_beta and welfare_diff <= tol_welfare:
    status = "passed"
else:
    status = "FAILED"
    raise AssertionError("Baseline cross-check failed")

print(f"Baseline cross-check {status}: Δbar_beta_opt={beta_diff:.3e}, Δwelfare_pct_change={welfare_diff:.3e}")
print(f"Total wall time: {total_time:.2f} s; per-share avg: {total_time/len(shares):.2f} s")
print(f"{len(shares)} shares evaluated")
print(f"CSV saved to {csv_path}")
print(f"PNG saved to {png_path}")


In [ ]:

display(results_df.head(5))
display(results_df.tail(5))
